In [1]:
import sys
import os
sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../.."))
from create_fashion_items_image import create_cw_image
from fashion_class.ImageStruct import ImageStruct

In [2]:
import importlib
from fashion_class.CapsuleWardrobeClass import CapsuleWardrobe
from fashion_class.FashionItem import FashionItem
import gc
import random
from CreateCW import change_item_recommandation, create_cw, search_alternate_item

In [5]:
annotation_file, tensor_file = \
        'D:/M1/fashion/optimization/data/anotation_new.csv', 'D:/M1/fashion/optimization/data/resnet50_tensor.pt'
L = 100
LL = 10
for loop in range(L):
    # データセットを初期化
    dataset = ImageStruct(annotation_file, tensor_file, init_item_length=500)

    category_list = ["tops", "bottoms", "shoes"]
    category_dataset = {
        'tops': dataset.tops,
        'bottoms': dataset.bottoms,
        'shoes': dataset.shoes
    }
    required_items = {}
    initial_items = {}
    for c in category_list:
        initial_items[c] = category_dataset[c][:4]
    cw = create_cw(required_items, dataset, initial_items)
    for i in range(LL):
        gc.collect()
        original_cw_score = cw.get_score()
        kind, index = random.choice(['tops', 'bottoms', 'shoes']), random.choice(list(range(4)))
        new_item = search_alternate_item(cw, kind, index-1, dataset)
        # 1枚多く計算する
        # 既存のアイテムは外さない
        required_items = {
            'tops': cw.get_tops(),
            'bottoms':cw.get_bottoms(),
            'shoes':cw.get_shoes()
        }

        # 既存のアイテムのお気に入りの入れ替え
        required_items[kind][index-1] = new_item
        cw_replacement = CapsuleWardrobe(initial_items=required_items)
        cw_replacement_score = cw_replacement.get_score()

        alternate_cw = create_cw(required_items, dataset, max_length=cw.max_length + 1)
        new_items = {
            'tops': alternate_cw.get_tops()[-1],
            'bottoms': alternate_cw.get_bottoms()[-1],
            'shoes': alternate_cw.get_shoes()[-1]
        }
        recommended_change = change_item_recommandation(alternate_cw)
        for (r, i) in recommended_change.items():
            if r == kind or i+1 == 5:
                continue
            required_items[r][i] = new_items[r]
        cw = CapsuleWardrobe(initial_items=required_items)
        cw_changed_score = cw.get_score()
        with open('D:/M1/fashion/optimization/data/alternated_score.txt', 'a') as f:
            f.write(f'{original_cw_score}, {cw_replacement_score}, {cw_changed_score}\n')
    print(f"{ loop // L }%終了")

【==================================================================================================  】0%終了
【==================================================================================================  】0%終了
【==================================================================================================  】0%終了
【==================================================================================================  】0%終了
【==================================================================================================  】0%終了
【==================================================================================================  】0%終了
【==================================================================================================  】

KeyboardInterrupt: 